<H1><b>Pertemuan 6 - Recurrent Neural Networks</b></H1>

<H3>Nama : Akmal Zuhdy Prasetya</H3>
<H3>NIM  : H071191035</H3>
<H5>Berdasarkan: <a>http://d2l.ai/chapter_recurrent-neural-networks/index.html<a><H5>